In [ ]:
import os
import sys
import importlib.util
import pandas as pd
import re

In [ ]:
txts_cleaned_path = "/datasets/txts-cleaned"
csv_cleaned_path = "outputs/csvs"

In [ ]:
!mkdir -p $csv_cleaned_path

In [ ]:
# create a dataframe to populate and then generate csv
dataset = pd.DataFrame(
    columns=["filename", "case_number", "client_txt", "therapist_txt"]
)

In [ ]:
# loading the dataset from txt file
def merge_convos(txts, prompt):
    if len(txts) == 1:
        return txts[0]
    # remove all 'CLIENT: ' beginnings, merge all text with space and create new convo with 'CLIENT: ' prompt
    convos = [txt.replace(prompt, "").strip() for txt in txts]
    convos = " ".join(convos)
    convos = re.sub("\n", " ", convos)
    convos = prompt + convos
    return convos


for filename in os.listdir(txts_cleaned_path):
    file_path = f"{txts_cleaned_path}/{filename}"
    case_number = 0
    with open(file_path, "r") as file:
        client_record, therapist_record, client_txts, therapist_txts = (
            False,
            False,
            [],
            [],
        )
        for line in file:
            line = line.replace("\n", "")
            # print(line.encode('unicode_escape'))
            if line == "BEGIN TRANSCRIPT":
                case_number += 1
                client_record, therapist_record, client_txts, therapist_txts = (
                    False,
                    False,
                    [],
                    [],
                )
                continue
            if line == "END TRANSCRIPT":
                continue
            if re.match("^CLIENT: ", line):
                # check if therapist reply recorded
                if therapist_record:
                    client_convos = merge_convos(client_txts, "CLIENT: ")
                    therapist_convos = merge_convos(therapist_txts, "THERAPIST: ")
                    dataset.loc[len(dataset)] = [
                        filename,
                        case_number,
                        client_convos,
                        therapist_convos,
                    ]
                    client_record, therapist_record, client_txts, therapist_txts = (
                        False,
                        False,
                        [],
                        [],
                    )
                client_record = True
                client_txts.append(line)
                continue
            if re.match("^THERAPIST: ", line):
                # ignore conversation by therapist unless client record started
                if not client_record:
                    continue
                # register reply by therapist
                therapist_record = True
                therapist_txts.append(line)
                continue

In [ ]:
len(dataset)

In [ ]:
dataset[0:30]

In [ ]:
dataset.to_csv(f"{csv_cleaned_path}/data.csv", index=False)